In [ ]:
import sys
sys.path.append("..")

import random
import math
import itertools
from copy import deepcopy
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union, Dict

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
import plotly.graph_objects as go
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
from sklearn.manifold import TSNE
from sklearn.decomposition import IncrementalPCA

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
import torchaudio.transforms as AT
import torchaudio.functional as AF
from torchvision.utils import make_grid
from IPython.display import display, Audio
import torchaudio
from torchaudio.io import StreamReader

from src.datasets import *
from src.algo import GreedyLibrary
from src.util.image import *
from src.util import to_torch_device, iter_batches
from src.patchdb import PatchDB, PatchDBIndex
from src.models.encoder import *
from src.models.cnn import *
from src.models.util import *
from src.util.audio import *
from src.util.files import *
from src.util.embedding import *
from scripts import datasets
from src.algo import AudioUnderstander 

def resize(img, scale: float, mode: VF.InterpolationMode = VF.InterpolationMode.NEAREST):
    return VF.resize(img, [max(1, int(s * scale)) for s in img.shape[-2:]], mode, antialias=False)

In [ ]:
wave, rate = torchaudio.load("/home/bergi/Music/olli/24.07.19 eberhardt finaaaaaal.wav")
wave.shape, rate

In [ ]:
wave_slice = wave[:1, rate:rate * 2]
print(wave_slice.shape)
speccer = AT.MelSpectrogram(
    n_mels=128,
    n_fft=2048,
    win_length=rate // 200,
    #hop_length=220,
    power=1.,
    f_max=rate // 2,
    sample_rate=rate,
)

spec = speccer(wave_slice)
print(spec.shape)
display(VF.to_pil_image(VF.vflip(spec) / spec.max()))
display(Audio(wave_slice, rate=rate))

In [ ]:
class SpecToWave(nn.Module):
    
    def __init__(
            self, 
            spec_shape: Tuple[int, int],
            out_size: int,
            n_bands: int,
    ):
        super().__init__()
        self.spec_shape = tuple(spec_shape)
        self.out_size = out_size
        self.n_bands = n_bands
        self.conv = Conv1dBlock((spec_shape[0], spec_shape[0], 1), kernel_size=5)
        out_shape = self.conv.get_output_shape(self.spec_shape)
        self.linear = nn.Linear(math.prod(out_shape), self.n_bands * 3)
        
    def forward(self, spec: torch.Tensor) -> torch.Tensor:
        if spec.ndim == 2:
            spec = spec.unsqueeze(0)
        bs = spec.shape[0]
        
        c = self.conv(spec)
        b = self.linear(c).view(-1, self.n_bands, 3)
        
        t = torch.linspace(0, 1, self.out_size).to(c.dtype).to(c.device).view(1, 1, -1).expand(bs, self.n_bands, -1)
        
        freq = b[..., 0].unsqueeze(-1) * 100
        phase = b[..., 1].unsqueeze(-1) 
        amp = b[..., 2].unsqueeze(-1)
        wave = torch.sin((t * freq + phase) * 6.28) * amp
        return wave.mean(-2)

model = SpecToWave(spec.shape[-2:], 10000, 10)
print(f"params: {num_module_parameters(model):,}")
with torch.no_grad():
    audio = model(spec)
print(audio.shape)
display(plot_audio(audio, (128, 128)))
Audio(audio, rate=rate)

In [ ]:
20_000 * 128 * 128 // 1024**2